In [1]:
import os
import time
import logging
import yaml
import ast
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from pprgo import utils
from pprgo import ppr
from pprgo import pprgo

In [2]:
# Set up logging
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
        fmt='%(asctime)s (%(levelname)s): %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel('INFO')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Download dataset

# Load config

In [3]:
with open('config_demo.yaml', 'r') as c:
    config = yaml.safe_load(c)

In [4]:
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [5]:
data_file           = config['data_file']           # Path to the .npz data file
data_file = 'data/cora_full.npz'
split_seed          = config['split_seed']          # Seed for splitting the dataset into train/val/test
ntrain_div_classes  = config['ntrain_div_classes']  # Number of training nodes divided by number of classes
ntrain_div_classes = 20
attr_normalization  = config['attr_normalization']  # Attribute normalization. Not used in the paper

alpha               = config['alpha']               # PPR teleport probability
alpha = 0.25
eps                 = config['eps']                 # Stopping threshold for ACL's ApproximatePR
topk                = config['topk']                # Number of PPR neighbors for each node
ppr_normalization   = config['ppr_normalization']   # Adjacency matrix normalization for weighting neighbors

hidden_size         = config['hidden_size']         # Size of the MLP's hidden layer
nlayers             = config['nlayers']             # Number of MLP layers
weight_decay        = config['weight_decay']        # Weight decay used for training the MLP
dropout             = config['dropout']             # Dropout used for training

lr                  = config['lr']                  # Learning rate
max_epochs          = config['max_epochs']          # Maximum number of epochs (exact number if no early stopping)
batch_size          = config['batch_size']          # Batch size for training
batch_mult_val      = config['batch_mult_val']      # Multiplier for validation batch size

eval_step           = config['eval_step']           # Accuracy is evaluated after every this number of steps
run_val             = config['run_val']             # Evaluate accuracy on validation set during training

early_stop          = config['early_stop']          # Use early stopping
patience            = config['patience']            # Patience for early stopping

nprop_inference     = config['nprop_inference']     # Number of propagation steps during inference
inf_fraction        = config['inf_fraction']        # Fraction of nodes for which local predictions are computed during inference

# Load the data

In [6]:
start = time.time()
(adj_matrix, attr_matrix, labels,
 train_idx, val_idx, test_idx) = utils.get_data(
        f"{data_file}",
        seed=split_seed,
        ntrain_div_classes=ntrain_div_classes,
        normalize_attr=attr_normalization
)
try:
    d = attr_matrix.n_columns
except AttributeError:
    d = attr_matrix.shape[1]
nc = labels.max() + 1
print(nc)

print('total: ', adj_matrix.shape[0])
print('Training: ', len(train_idx))
print('Validation: ', len(val_idx))
print('Testing: ', len(test_idx))

time_loading = time.time() - start
print(f"Runtime: {time_loading:.2f}s")

70
total:  18800
Training:  3150
Validation:  15650
Testing:  0
Runtime: 0.26s


# Preprocessing: Calculate PPR scores

In [7]:
# compute the ppr vectors for train/val nodes using ACL's ApproximatePR
start = time.time()
topk_train = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, train_idx, topk,
                                 normalization=ppr_normalization)
if run_val:
    topk_val = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, val_idx, topk,
                                   normalization=ppr_normalization)
else:
    topk_val = None

print(topk_train.shape)
time_preprocessing = time.time() - start
print(f"Runtime: {time_preprocessing:.2f}s")

(3150, 18800)
Runtime: 11.90s


# Training: Set up model and train

In [8]:
start = time.time()
tf.reset_default_graph()
tf.set_random_seed(0)

model = pprgo.PPRGo(d, nc, hidden_size, nlayers, lr, weight_decay, dropout,
                    sparse_features=type(attr_matrix) is not np.ndarray)

sess = tf.compat.v1.Session()
with sess.as_default():
    tf.compat.v1.global_variables_initializer().run()
    nepochs, loss_hist, acc_hist, f1_hist = pprgo.train(
            sess=sess, model=model, attr_matrix=attr_matrix,
            train_idx=train_idx, val_idx=val_idx,
            topk_train=topk_train, topk_val=topk_val,
            labels=labels,
            max_epochs=max_epochs, batch_size=batch_size, batch_mult_val=batch_mult_val,
            eval_step=eval_step, early_stop=early_stop, patience=patience)
time_training = time.time() - start
logging.info('Training done.')
print(f"Runtime: {time_training:.2f}s")

2022-06-02 17:47:19 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_9722/2975614973.py:2: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.



2022-06-02 17:47:19 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_9722/2975614973.py:3: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

2022-06-02 17:47:21 (INFO): Epoch 2, step 20: train 4.09662
2022-06-02 17:47:22 (INFO): Epoch 5, step 40: train 3.81260
2022-06-02 17:47:23 (INFO): Epoch 8, step 60: train 3.52556
2022-06-02 17:47:24 (INFO): Epoch 11, step 80: train 3.27781
2022-06-02 17:47:25 (INFO): Epoch 14, step 100: train 2.75892
2022-06-02 17:47:26 (INFO): Epoch 17, step 120: train 2.53977
2022-06-02 17:47:26 (INFO): Epoch 19, step 140: train 1.44046
2022-06-02 17:47:27 (INFO): Epoch 22, step 160: train 2.32340
2022-06-02 17:47:28 (INFO): Epoch 25, step 180: train 2.03010
2022-06-02 17:47:29 (INFO): Epoch 28, step 200: train 1.90088
2022-06-02 17:47:30 (INFO): Epoch 31, step 220: train 1.87701
2022-06-02 17:47:30 (INFO): Epoch 34, step 240: train 1.55377
2022-06-02 17:47:31 (INFO): Epoch 37, step 260: train

Runtime: 62.13s


# Inference (val and test)

In [9]:
start = time.time()
predictions, time_logits, time_propagation = model.predict(
        sess=sess, adj_matrix=adj_matrix, attr_matrix=attr_matrix, alpha=alpha,
        nprop=nprop_inference, inf_fraction=inf_fraction,
        ppr_normalization=ppr_normalization)
time_inference = time.time() - start
print(f"Runtime: {time_inference:.2f}s")

Runtime: 0.19s


# Collect and print results

In [10]:
acc_train = 100 * accuracy_score(labels[train_idx], predictions[train_idx])
acc_val = 100 * accuracy_score(labels[val_idx], predictions[val_idx])
acc_test = 100 * accuracy_score(labels[test_idx], predictions[test_idx])

print('len: ', len(train_idx), train_idx)
print('len: ', len(val_idx), val_idx)
print(predictions.shape)
print(test_idx)
print(predictions[test_idx].shape)

print(predictions[0])
f1_train = f1_score(labels[train_idx], predictions[train_idx], average='macro')
f1_val = f1_score(labels[val_idx], predictions[val_idx], average='macro')
f1_test = f1_score(labels[test_idx], predictions[test_idx], average='macro')

#gpu_max_bytes = tf.contrib.memory_stats.MaxBytesInUse()
#gpu_memory = sess.run(gpu_max_bytes)
memory = utils.get_max_memory_bytes()

time_total = time_preprocessing + time_training + time_inference

len:  3150 [    8    12    16 ... 18793 18796 18798]
len:  15650 [    0     1     2 ... 18795 18797 18799]
(18800,)
[]
(0,)
48


/Users/arielo97/Documents/University-TOPUSH/Intership/Experiments/internship_env/lib/python3.7/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/arielo97/Documents/University-TOPUSH/Intership/Experiments/internship_env/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/arielo97/Documents/University-TOPUSH/Intership/Experiments/internship_env/lib/python3.7/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/arielo97/Documents/University-TOPUSH/Intership/Experiments/internship_env/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
print(f'''
Accuracy: Train: {acc_train:.1f}%, val: {acc_val:.1f}%, test: {acc_test:.1f}%
F1 score: Train: {f1_train:.3f}, val: {f1_val:.3f}, test: {f1_test:.3f}

Runtime: Preprocessing: {time_preprocessing:.2f}s, training: {time_training:.2f}s, inference: {time_inference:.2f}s -> total: {time_total:.2f}s
Memory: Main: {(memory/1024) / 2**30:.2f}GB
''')

#Memory: Main: {memory / 2**30:.2f}GB, GPU: {gpu_memory / 2**30:.3f}GB


Accuracy: Train: 97.5%, val: 67.1%, test: nan%
F1 score: Train: 0.952, val: 0.588, test: nan

Runtime: Preprocessing: 11.90s, training: 62.13s, inference: 0.19s -> total: 74.22s
Memory: Main: 0.57GB

